# **Machine Learning: Project Part 2**

---

**Author: Damien Farrell**

---

In [135]:
# Imports

import pandas as pd
import seaborn as sns

sns.set_theme()

In [2]:
# Environment Variables
load_dotenv()
# HF_API_KEY = os.getenv("HF_API_KEY")
# assert HF_API_KEY, "Please set your Hugging Face API key in the HF_API_KEY environment variable."

# Audio File Path
# AUDIO_FILE_PATH = "./audio/TrumpHarrisDebate.wav"
# assert AUDIO_FILE_PATH, "Please place your audio in a folder called 'audio'. Note: use backslashes if on Windows"

## **Project Part 2: Seal Call Discrimination**

### Objective
The aim of this project is to analyze a recorded dataset to investigate the feasibility of discriminating between different seal calls. The project is structured in steps to build a machine learning model that can potentially detect seal calls from audio recordings. While creating a fully functional detector is the ultimate goal, it may not be the final outcome of this project.

### Key Steps

#### Step 1: Data Pre-processing and Management
1. **Dataset Preparation**:
   - Work with provided uncompressed `.wav` files, which may contain large portions without any seal calls.
   - Create a dataset from annotated data using the provided Jupyter notebook.
   
2. **Spectrogram Extraction**:
   - Extract spectrograms for each seal call, ensuring consistent size by determining the longest call (in time) and the broadest in frequency.
   - Save spectrograms as raw 2D arrays (not images) with metadata.

3. **Handling No-Call Data**:
   - Create "no-call" spectrograms from unannotated regions of recordings, ensuring they match the frequency region of call spectrograms.

COMPLETE IN OTHER NOTEBOOK `data_preprocessing.ipynb` !!!!!!!!!!!!!!!!!!!!!!

---

#### Step 2: Model Training
1. **Dataset Utilisation**:
   - Use the prepared spectrogram dataset to train the machine learning model.

3. **Evaluation**:
   - Test the model’s performance on a separate test dataset.

---

#### Step 3: Refine
1. **Parameter Tuning**:
   - Adjust parameters in the data extraction/spectrogram creation process (e.g., `nfft`, `noverlap`).
   - Manage computational costs by splitting `.wav` files using tools like `pydub`.

2. **Validation**:
   - Apply the Rupe B/No Call classifier on an entire `.wav` file (held back during training).
   - Ensure no false positives due to the presence of other annotated calls.

---

---

### **1. Performs Speaker Diarisation Analysis** 

---

---

### **2. Performs Speech to Text Analysis** 

---

---

### **4. Testing & Evaluation**

---

---

### **References**

1. [Bird Song Dataset on Kaggle](https://www.kaggle.com/code/sophiagnetneva/cnn-for-sound-classification-bird-calls-90)





---

# END